In [1]:
import pandas as pd
import numpy as np

In [2]:
def reformat_date(df):
    appoint_date = []
    for i in range(len(df)):
        day = df[i]
        if pd.isna(day):
            appoint_date.append(np.nan)  
        elif day == 'REDACTED':
            appoint_date.append(np.nan)  
        elif len(day) == 8:
            if int(day[6:8])+2000 > 2021:
                appoint_date.append('19'+day[6:8]+'-'+day[0:5])
            else: 
                appoint_date.append('20'+day[6:8]+'-'+day[0:5] )
        else:
            mon = day[5:8]
            if mon == 'JAN':
                appoint_date.append(day[0:5]+'01'+day[8:])
            elif mon == 'FEB':
                appoint_date.append(day[0:5]+'02'+day[8:])
            elif mon == 'MAR':
                appoint_date.append(day[0:5]+'03'+day[8:])
            elif mon == 'APR':
                appoint_date.append(day[0:5]+'04'+day[8:])
            elif mon == 'MAY':
                appoint_date.append(day[0:5]+'05'+day[8:])
            elif mon == 'JUN':
                appoint_date.append(day[0:5]+'06'+day[8:])
            elif mon == 'JUL':
                appoint_date.append(day[0:5]+'07'+day[8:])
            elif mon == 'AUG':
                appoint_date.append(day[0:5]+'08'+day[8:])
            elif mon == 'SEP':
                appoint_date.append(day[0:5]+'09'+day[8:])
            elif mon == 'OCT':
                appoint_date.append(day[0:5]+'10'+day[8:])
            elif mon == 'NOV':
                appoint_date.append(day[0:5]+'11'+day[8:])
            elif mon == 'DEC':
                appoint_date.append(day[0:5]+'12'+day[8:])
    return appoint_date

In [3]:
df_trr = pd.read_csv("trr_trr_refresh.csv")
df_trr['trr_datetime'] = pd.to_datetime(df_trr['trr_datetime'])
df_trr['beat'] = df_trr['beat'].astype(int)
df_trr['officer_appointed_date'] = pd.to_datetime(reformat_date(df_trr['officer_appointed_date']))
df_trr['officer_birth_year'] = df_trr['officer_birth_year'].fillna(0.0).astype(int)
df_trr['officer_birth_year'].replace(0, np.nan,inplace=True)
df_trr['officer_age'] = df_trr['officer_age'].astype(int)
df_trr['officer_on_duty'] = df_trr['officer_on_duty'].astype(bool)
df_trr['officer_injured'] = df_trr['officer_injured'].astype(bool)
df_trr['officer_in_uniform'] = df_trr['officer_in_uniform'].astype(bool)
df_trr['subject_birth_year'] = df_trr['subject_birth_year'].astype(int)
df_trr['subject_age'] = df_trr['subject_age'].astype(int)
df_trr['subject_armed'] = df_trr['subject_armed'].astype(bool)
df_trr['subject_injured'] = df_trr['subject_injured'].astype(bool)
df_trr['subject_alleged_injury'] = df_trr['subject_alleged_injury'].astype(bool)
df_trr['notify_oemc'] = df_trr['notify_oemc'].astype(bool)
df_trr['notify_district_sergeant'] = df_trr['notify_district_sergeant'].astype(bool)
df_trr['notify_op_command'] = df_trr['notify_op_command'].astype(bool)
df_trr['notify_det_division'] = df_trr['notify_det_division'].astype(bool)
df_trr['trr_created'] = pd.to_datetime(df_trr['trr_created'])

/Users/yj/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_weapondischarge = pd.read_csv("trr_weapondischarge_refresh.csv")
df_weapondischarge['firearm_reloaded'] = df_weapondischarge['firearm_reloaded'].astype(bool)
df_weapondischarge['sight_used'] = df_weapondischarge['sight_used'].astype(bool)

In [5]:
df_trrstatus = pd.read_csv("trr_trrstatus_refresh.csv")
df_trrstatus['officer_appointed_date'] = pd.to_datetime(reformat_date(df_trrstatus['officer_appointed_date']))
df_trrstatus['officer_birth_year'] = df_trrstatus['officer_birth_year'].fillna(0).astype(int)
df_trrstatus['officer_birth_year'].replace(0, np.nan,inplace=True)
df_trrstatus['status_datetime'] = pd.to_datetime(df_trrstatus['status_datetime'])

In [6]:
def reconciliation_race(df):
    race_map = dict({'AMER IND/ALASKAN NATIVE':'NATIVE AMERICAN/ALASKAN NATIVE',
                'AMER INDIAN / ALASKAN NATIVE':'NATIVE AMERICAN/ALASKAN NATIVE',
                'ASIAN / PACIFIC ISLANDER':'ASIAN/PACIFIC ISLANDER',
                'ASIAN/PACIFIC ISLANDER':'ASIAN/PACIFIC ISLANDER',
                'BLACK':'BLACK',
                'UNKNOWN':'NULL',
                 'UNKNOWN / REFUSED':'NULL',
                 'HISPANIC':'HISPANIC',
                 'WHITE':'WHITE'
                })
    for k in race_map:
        if k[0:2] == 'UN':
            df.replace(k, np.nan,inplace=True)
        else:
            df.replace(k,race_map[k] ,inplace=True)

reconciliation_race(df_trr['subject_race'])

In [11]:
def reconciliation_gender(df):
    race_map = dict({'FEMALE':'F',
                     'MALE':'M'
                })
    for k in race_map:
        df.replace(k,race_map[k] ,inplace=True)
        
reconciliation_gender(df_trr['subject_gender'])

In [12]:
def reconciliation_birth_year(df):
    birth_years = df.to_numpy()
    birth_years[birth_years < 100 ] += 1900
    birth_years[birth_years < 200 ] *= 10
    for i in range(1000,2000,100):
        birth_years[birth_years < i ] += 2000-i   
    df = pd.DataFrame(birth_years)

reconciliation_birth_year(df_trr['subject_birth_year'])

In [14]:
# Indoor/outdoor change specific column.
df_trr.loc[(df_trr.indoor_or_outdoor == 'OUTDOOR'), 'indoor_or_outdoor'] = 'Outdoor'
df_trr.loc[(df_trr.indoor_or_outdoor == 'INDOOR'), 'indoor_or_outdoor'] = 'Indoor'

# Street. 1. change to camel case
df_trr['street'] = df_trr['street'].str.title()

# location. 1. change to camel case 2. change specific items to the correct form.
df_trr['location'] = df_trr['location'].str.title()

df_trr.loc[(df_trr.location == 'Cha Hallway / Stairwell / Elevator'), 'location'] = 'Cha Hallway/Stairwell/Elevator'
df_trr.loc[(df_trr.location == 'Cha Parking Lot / Grounds'), 'location'] = 'Cha Parking Lot/Grounds'
df_trr.loc[(df_trr.location == 'Church / Synagogue / Place Of Worship'), 'location'] = 'Church/Synagogue/Place Of Worship'
df_trr.loc[(df_trr.location == 'College / University - Grounds'), 'location'] = 'College/University Grounds'
df_trr.loc[(df_trr.location == 'Factory / Manufacturing Building'), 'location'] = 'Factory/Manufacturing Building'
df_trr.loc[(df_trr.location == 'Government Building / Property'), 'location'] = 'Government Building/Property'
df_trr.loc[(df_trr.location == 'Highway / Expressway'), 'location'] = 'Highway/Expressway'
df_trr.loc[(df_trr.location == 'Hospital Building / Grounds'), 'location'] = 'Hospital Building/Grounds'
df_trr.loc[(df_trr.location == 'Hotel / Motel'), 'location'] = 'Hotel/Motel'
df_trr.loc[(df_trr.location == 'Movie House / Theater'), 'location'] = 'Movie House/Theater'
df_trr.loc[(df_trr.location == 'Nursing / Retirement Home'), 'location'] = 'Nursing Home/Retirement Home'
df_trr.loc[(df_trr.location == 'Other (Specify)'), 'location'] = 'Other'
df_trr.loc[(df_trr.location == 'Other Railroad Property / Train Depot'), 'location'] = 'Other Railroad Prop / Train Depot'
df_trr.loc[(df_trr.location == 'Parking Lot / Garage (Non Residential)'), 'location'] = 'Parking Lot/Garage(Non.Resid.)'
df_trr.loc[(df_trr.location == 'Other Railroad Property / Train Depot'), 'location'] = 'Other Railroad Prop / Train Depot'
df_trr.loc[(df_trr.location == 'Police Facility / Vehicle Parking Lot'), 'location'] = 'Police Facility/Veh Parking Lot'
df_trr.loc[(df_trr.location == 'Residence - Porch / Hallway'), 'location'] = 'Residence Porch/Hallway'
df_trr.loc[(df_trr.location == 'Residence - Garage'), 'location'] = 'Residence-Garage'
df_trr.loc[(df_trr.location == 'Residence - Yard (Front / Back)'), 'location'] = 'Residential Yard (Front/Back)'
df_trr.loc[(df_trr.location == 'School - Private Building'), 'location'] = 'School, Private, Building'
df_trr.loc[(df_trr.location == 'School - Private Grounds'), 'location'] = 'School, Private, Grounds'
df_trr.loc[(df_trr.location == 'School - Public Building'), 'location'] = 'School, Public, Building'
df_trr.loc[(df_trr.location == 'School - Public Grounds'), 'location'] = 'School, Public, Grounds'
df_trr.loc[(df_trr.location == 'Sports Arena / Stadium'), 'location'] = 'Sports Arena/Stadium'
df_trr.loc[(df_trr.location == 'Tavern / Liquor Store'), 'location'] = 'Tavern/Liquor Store'
df_trr.loc[(df_trr.location == 'Vacant Lot / Land'), 'location'] = 'Vacant Lot/Land'
df_trr.loc[(df_trr.location == 'Vehicle - Other Ride Share Service (Lyft, Uber, Etc.)'), 'location'] = 'Vehicle - Other Ride Service'
df_trr.loc[(df_trr.location == 'Vehicle - Commercial'), 'location'] = 'Vehicle-Commercial'
df_trr.loc[(df_trr.location == 'Cta Parking Lot / Garage / Other Property'), 'location'] = 'Cta Garage / Other Property'
df_trr.loc[(df_trr.location == 'Lakefront / Waterfront / Riverbank'), 'location'] = 'Lakefront/Waterfront/Riverbank'
df_trr.loc[(df_trr.location == 'Medical / Dental Office'), 'location'] = 'Medical/Dental Office'
df_trr.loc[(df_trr.location == 'Airport Parking Lot'), 'location'] = 'Airport/Aircraft'
df_trr.loc[(df_trr.location == 'Airport Terminal Mezzanine - Non-Secure Area'), 'location'] = 'Airport Terminal Lower Level - Non-Secure Area'


# birth year
def age_sanity(row):
    if row < 100:
        return 1900 + row
    if 100 <= row < 1900:
        return 1900 + row % 100
    return row

df_trr.subject_birth_year = df_trr.subject_birth_year.apply(age_sanity)


In [23]:
# Read weapon file.
sub_weapon_refresh = pd.read_csv("trr_subjectweapon_refresh.csv")
# sub_weapon = pd.read_csv("../../trr_subjectweapon.csv")

# Weapon, change specific items to the correct form.
sub_weapon_refresh.loc[(sub_weapon_refresh.weapon_type == 'CHEMICAL WEAPON') | (sub_weapon_refresh.weapon_type == 'TASER / STUN GUN'), 'weapon_type'] = 'OTHER (SPECIFY)'
sub_weapon_refresh.loc[(sub_weapon_refresh.weapon_type == 'VEHICLE'), 'weapon_type'] = 'VEHICLE - ATTEMPTED TO STRIKE OFFICER WITH VEHICLE'


In [25]:
df_trr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17465 entries, 0 to 17464
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        17465 non-null  int64         
 1   event_number              17213 non-null  float64       
 2   cr_number                 1614 non-null   object        
 3   beat                      17465 non-null  int64         
 4   block                     17465 non-null  object        
 5   direction                 17438 non-null  object        
 6   street                    17465 non-null  object        
 7   location                  17465 non-null  object        
 8   trr_datetime              17465 non-null  object        
 9   indoor_or_outdoor         17196 non-null  object        
 10  lighting_condition        17193 non-null  object        
 11  weather_condition         16912 non-null  object        
 12  notify_oemc       